<a href="https://colab.research.google.com/github/pawnzo90z/competitions/blob/master/wns_challange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os 
os.chdir("/content/drive/My Drive/Colab Notebooks/for_google_notebook/WNS/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import warnings
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import numpy as np
from tqdm import tqdm
from scipy import stats



train = pd.read_csv('train/train.csv')
test = pd.read_csv('test.csv')

item_data = pd.read_csv('train/item_data.csv')
view_data = pd.read_csv('train/view_log.csv')

In [ ]:
train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [ ]:
item_data.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


In [ ]:
view_data.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


In [ ]:
view_data['server_time'] = pd.to_datetime(view_data['server_time'])
view_data = view_data.sort_values('server_time').reset_index(drop = True)

train['impression_time'] = pd.to_datetime(train['impression_time'])
test['impression_time'] = pd.to_datetime(test['impression_time'])

train = train.sort_values('impression_time').reset_index(drop = True)
test = test.sort_values('impression_time').reset_index(drop = True)

ltr = len(train)

data = pd.concat([train, test], axis = 0).reset_index(drop = True)





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


In [ ]:
data['user_id'].map(data.groupby('user_id')['impression_time'].apply(lambda x: np.nanmean(x.diff() / np.timedelta64(1, 's'))).to_dict())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


0          74670.000000
1         113920.800000
2          75219.473684
3            540.000000
4          46505.000000
              ...      
328279              NaN
328280              NaN
328281    129725.454545
328282              NaN
328283     89163.750000
Name: user_id, Length: 328284, dtype: float64

In [ ]:
data['user_id']

0         87862
1         89464
2         58442
3          4238
4         63410
          ...  
328279     3547
328280    56499
328281    29427
328282    75373
328283    82078
Name: user_id, Length: 328284, dtype: int64

In [ ]:
data['diff_time_mean'] = data['user_id'].map(data.groupby('user_id')['impression_time'].apply(lambda x: np.nanmean(x.diff() / np.timedelta64(1, 's'))).to_dict())
data['diff_time_max'] = data['user_id'].map(data.groupby('user_id')['impression_time'].apply(lambda x: np.nanmax(x.diff() / np.timedelta64(1, 's'))).to_dict())
data['diff_time_min'] = data['user_id'].map(data.groupby('user_id')['impression_time'].apply(lambda x: np.nanmin(x.diff() / np.timedelta64(1, 's'))).to_dict())





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: All-NaN axis encountered
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: All-NaN axis encountered
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data['os_version'] = data['os_version'].astype('category').cat.codes

In [ ]:
data.head()

,app_code,impression_id,impression_time,is_4G,is_click,os_version,user_id,diff_time_mean,diff_time_max,diff_time_min,cnt_unique_app
0,422,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,0,0.0,2,87862,74670.000000,148200.0,1140.0,1
1,129,c81e728d9d4c2f636f067f89cc14862c,2018-11-15 00:00:00,0,0.0,0,89464,113920.800000,347700.0,180.0,1
2,127,eccbc87e4b5ce2fe28308fd9f2a7baf3,2018-11-15 00:00:00,0,0.0,1,58442,75219.473684,259620.0,8400.0,1
3,371,a87ff679a2f3e71d9181a67b7542122c,2018-11-15 00:00:00,0,0.0,1,4238,540.000000,540.0,540.0,1
4,467,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,1,1.0,1,63410,46505.000000,169800.0,240.0,2


In [ ]:
data[data['user_id'] == 63410]['app_code'].value_counts()

467    50
283    11
Name: app_code, dtype: int64

In [ ]:
data['cnt_unique_app'] = data['user_id'].map(data.groupby('user_id')['app_code'].apply(lambda x: x.unique().size).to_dict())

In [ ]:
data.head()

,app_code,impression_id,impression_time,is_4G,is_click,os_version,user_id,diff_time_mean,diff_time_max,diff_time_min,cnt_unique_app
0,422,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,0,0.0,2,87862,74670.000000,148200.0,1140.0,1
1,129,c81e728d9d4c2f636f067f89cc14862c,2018-11-15 00:00:00,0,0.0,0,89464,113920.800000,347700.0,180.0,1
2,127,eccbc87e4b5ce2fe28308fd9f2a7baf3,2018-11-15 00:00:00,0,0.0,1,58442,75219.473684,259620.0,8400.0,1
3,371,a87ff679a2f3e71d9181a67b7542122c,2018-11-15 00:00:00,0,0.0,1,4238,540.000000,540.0,540.0,1
4,467,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,1,1.0,1,63410,46505.000000,169800.0,240.0,2


In [ ]:
data['hour'] = data['impression_time'].dt.hour
data['minute'] = data['impression_time'].dt.minute

In [ ]:
for col in ['app_code', 'user_id']:
    data['vc_' + col] = data[col].map(data[col].value_counts().to_dict())
data.head()

,app_code,impression_id,impression_time,is_4G,is_click,os_version,user_id,diff_time_mean,diff_time_max,diff_time_min,cnt_unique_app,hour,minute,vc_app_code,vc_user_id
0,422,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,0,0.0,2,87862,74670.000000,148200.0,1140.0,1,0,0,535,3
1,129,c81e728d9d4c2f636f067f89cc14862c,2018-11-15 00:00:00,0,0.0,0,89464,113920.800000,347700.0,180.0,1,0,0,9397,26
2,127,eccbc87e4b5ce2fe28308fd9f2a7baf3,2018-11-15 00:00:00,0,0.0,1,58442,75219.473684,259620.0,8400.0,1,0,0,14059,39
3,371,a87ff679a2f3e71d9181a67b7542122c,2018-11-15 00:00:00,0,0.0,1,4238,540.000000,540.0,540.0,1,0,0,12061,2
4,467,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,1,1.0,1,63410,46505.000000,169800.0,240.0,2,0,1,538,61


In [ ]:
view_data.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,876311,35913,15434
2,2018-10-15 08:58:00,android,585539,39617,22859
3,2018-10-15 08:58:00,android,264366,34712,11871
4,2018-10-15 08:58:00,android,941129,31754,127900


In [ ]:
item_data.head()

,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


In [ ]:
view_data = view_data.merge(item_data, how = 'left', on = 'item_id')

In [ ]:
view_data.head()

,server_time,device_type,session_id,user_id,item_id,item_price,category_1,category_2,category_3,product_type
0,2018-10-15 08:58:00,android,112333,4557,32970,54685.0,16.0,56.0,253.0,3184.0
1,2018-10-15 08:58:00,android,876311,35913,15434,6656.0,9.0,9.0,244.0,9008.0
2,2018-10-15 08:58:00,android,585539,39617,22859,2528.0,13.0,67.0,170.0,2874.0
3,2018-10-15 08:58:00,android,264366,34712,11871,2547.0,4.0,38.0,62.0,10412.0
4,2018-10-15 08:58:00,android,941129,31754,127900,3520.0,1.0,28.0,283.0,908.0


In [ ]:
data['app_code_cnt_unique_user'] = data['app_code'].map(data.groupby('app_code')['user_id'].apply(lambda x: x.unique().size).to_dict())

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
value_list = []
value_list_last = []
for user_id, imp_time in tqdm(zip(data.user_id.values, data.impression_time.values)):
    cur_data = data[(data.impression_time < imp_time) & (data.user_id == user_id)]
    value_list += [cur_data['is_click'].mean()]
    if cur_data.shape[0] > 0:
        value_list_last += [(imp_time - cur_data['impression_time'].values[-1]) /  np.timedelta64(1, 's')]
    else:
        value_list_last += [np.nan]
data['value_mean_user_id'] = value_list
data['diff_time_user_id_last'] = value_list_last



value_list = []
value_list_last = []
for app_code, imp_time in tqdm(zip(data.app_code.values, data.impression_time.values)):
    cur_data = data[(data.impression_time < imp_time) & (data.app_code == app_code)]
    value_list += [cur_data['is_click'].mean()]
    if cur_data.shape[0] > 0:
        value_list_last += [(imp_time - cur_data['impression_time'].values[-1]) /  np.timedelta64(1, 's')]
    else:
        value_list_last += [np.nan]

data['value_mean_app_code'] = value_list
data['diff_time_app_code_last'] = value_list_last



value_list_last = []
for app_code, imp_time in tqdm(zip(data.app_code.values, data.impression_time.values)):
    cur_data = data[(data.impression_time > imp_time) & (data.app_code == app_code)]
    if cur_data.shape[0] > 0:
        value_list_last += [(cur_data['impression_time'].values[0] - imp_time) /  np.timedelta64(1, 's')]
    else:
        value_list_last += [np.nan]
data['diff_time_app_code_next'] = value_list_last



value_list_last = []
for user_id, imp_time in tqdm(zip(data.user_id.values, data.impression_time.values)):
    cur_data = data[(data.impression_time > imp_time) & (data.user_id == user_id)]
    if cur_data.shape[0] > 0:
        value_list_last += [(cur_data['impression_time'].values[0] - imp_time) /  np.timedelta64(1, 's')]
    else:
        value_list_last += [np.nan]
data['diff_time_user_id_next'] = value_list_last





Streaming output truncated to the last 5000 lines.

64627it [03:53, 284.24it/s]


64657it [03:53, 288.33it/s]


64687it [03:53, 289.50it/s]


64716it [03:53, 285.01it/s]


64745it [03:53, 285.33it/s]


64774it [03:53, 282.41it/s]


64803it [03:53, 282.19it/s]


64832it [03:53, 278.57it/s]


64860it [03:54, 276.91it/s]


64888it [03:54, 271.21it/s]


64916it [03:54, 269.65it/s]


64944it [03:54, 270.25it/s]


64972it [03:54, 271.65it/s]


65000it [03:54, 272.80it/s]


65028it [03:54, 265.34it/s]


65056it [03:54, 269.20it/s]


65083it [03:54, 269.12it/s]


65111it [03:55, 270.52it/s]


65139it [03:55, 269.15it/s]


65168it [03:55, 273.85it/s]


65197it [03:55, 276.21it/s]


65225it [03:55, 274.41it/s]


65253it [03:55, 271.57it/s]


65281it [03:55, 273.24it/s]


65311it [03:55, 278.13it/s]


65339it [03:55, 268.61it/s]


65367it [03:55, 270.88it/s]


65395it [03:56, 272.58it/s]


65424it [03:56, 277.17it/s]


65454it [03:56, 280.99it/s]


65484it [03:56, 284.28it/s]


65513it [03:56, 28

In [ ]:
value_list_cnt = []
value_list_diff_mean = []
value_list_diff_max = []
value_list_diff_min = []

value_list_unique_session_id = []
value_list_unique_item_id = []
value_list_unique_category_1 = []
value_list_unique_category_2 = []
value_list_unique_category_3 = []
value_list_unique_product_type = []

value_list_mode_item_id = []
value_list_mode_category_1 = []
value_list_mode_category_2 = []
value_list_mode_category_3 = []
value_list_mode_product_type = []

for user_id, imp_time in tqdm(zip(data.user_id.values, data.impression_time.values)):
    cur_data = view_data[(view_data.server_time < imp_time) & (view_data.user_id == user_id)]
    value_list_cnt += [cur_data.shape[0]]
    
    if cur_data.shape[0] == 0:
        value_list_unique_session_id += [0]
        value_list_unique_item_id += [0]
        value_list_unique_category_1 += [0]
        value_list_unique_category_2 += [0]
        value_list_unique_category_3 += [0]
        value_list_unique_product_type += [0]

        value_list_mode_item_id += [-1]
        value_list_mode_category_1 += [-1]
        value_list_mode_category_2 += [-1]
        value_list_mode_category_3 += [-1]
        value_list_mode_product_type += [-1]
    
    else:
    
        value_list_unique_session_id += [cur_data['session_id'].unique().size]
        value_list_unique_item_id += [cur_data['item_id'].unique().size]
        value_list_unique_category_1 += [cur_data['category_1'].unique().size]
        value_list_unique_category_2 += [cur_data['category_2'].unique().size]
        value_list_unique_category_3 += [cur_data['category_3'].unique().size]
        value_list_unique_product_type += [cur_data['product_type'].unique().size]
        
        if len(cur_data['item_id'].value_counts()) > 0:
            value_list_mode_item_id += [cur_data['item_id'].value_counts().index[0]]
        else:
            value_list_mode_item_id += [-1]
            
        if len(cur_data['category_1'].value_counts()) > 0:
            value_list_mode_category_1 += [cur_data['category_1'].value_counts().index[0]]
        else:
            value_list_mode_category_1 += [-1]
        
        if len(cur_data['category_2'].value_counts()) > 0:
            value_list_mode_category_2 += [cur_data['category_2'].value_counts().index[0]]
        else:
            value_list_mode_category_2 += [-1]
        
        
        if len(cur_data['category_3'].value_counts()) > 0:
            value_list_mode_category_3 += [cur_data['category_3'].value_counts().index[0]]
        else:
            value_list_mode_category_3 += [-1]
        
        if len(cur_data['product_type'].value_counts()) > 0:
            value_list_mode_product_type += [cur_data['product_type'].value_counts().index[0]]
        else:
            value_list_mode_product_type += [-1]
    
    value_list_diff_mean += [cur_data['server_time'].diff().mean()]
    value_list_diff_max += [cur_data['server_time'].diff().max()]
    value_list_diff_min += [cur_data['server_time'].diff().min()]
    
data['value_cnt_view_user_id'] = value_list_cnt

data['user_id_unique_session_id'] = value_list_unique_session_id
data['user_id_unique_item_id'] = value_list_unique_item_id
data['user_id_unique_category_1'] = value_list_unique_category_1
data['user_id_unique_category_2'] = value_list_unique_category_2
data['user_id_unique_category_3'] = value_list_unique_category_3
data['user_id_unique_product_type'] = value_list_unique_product_type

data['user_id_mode_item_id'] = value_list_mode_item_id
data['user_id_mode_category_1'] = value_list_mode_category_1
data['user_id_mode_category_2'] = value_list_mode_category_2
data['user_id_mode_category_3'] = value_list_mode_category_3
data['user_id_mode_product_type'] = value_list_mode_product_type

data['value_diff_time_view_user_id_mean'] = value_list_diff_mean
data['value_diff_time_view_user_id_max'] = value_list_diff_max
data['value_diff_time_view_user_id_min'] = value_list_diff_min



data['value_diff_time_view_user_id_mean'] = data['value_diff_time_view_user_id_mean'] / np.timedelta64(1, 's')
data['value_diff_time_view_user_id_max'] = data['value_diff_time_view_user_id_max'] / np.timedelta64(1, 's')
data['value_diff_time_view_user_id_min'] = data['value_diff_time_view_user_id_min'] / np.timedelta64(1, 's')



cols = ['user_id_mode_item_id', 'user_id_mode_category_1',
       'user_id_mode_category_2', 'user_id_mode_category_3',
       'user_id_mode_product_type']

for col in cols:
    data['vc_' + col] = data[col].map(data[col].value_counts().to_dict())



data['gg_1_diff'] = data['app_code'].map(data.groupby('app_code')['user_id'].apply(lambda x: len(x) - x.unique().size).to_dict())
data['gg_1_ratio'] = data['app_code'].map(data.groupby('app_code')['user_id'].apply(lambda x: x.unique().size / len(x)).to_dict())

data['gg_2_diff'] = data['user_id'].map(data.groupby('user_id')['app_code'].apply(lambda x: len(x) - x.unique().size).to_dict())
data['gg_2_ratio'] = data['user_id'].map(data.groupby('user_id')['app_code'].apply(lambda x: x.unique().size / len(x)).to_dict())



train_cols = ['is_4G',
       'os_version',  'diff_time_mean', 'diff_time_max',
       'diff_time_min', 'cnt_unique_app', 'hour', 'minute',
       'vc_app_code', 'vc_user_id', 'app_code_cnt_unique_user',
       'value_mean_user_id', 'diff_time_user_id_last', 'value_mean_app_code',
       'diff_time_app_code_last', 'diff_time_app_code_next',
       'diff_time_user_id_next', 'value_cnt_view_user_id',
       'user_id_unique_session_id', 'user_id_unique_item_id',
       'user_id_unique_category_1', 'user_id_unique_category_2',
       'user_id_unique_category_3', 'user_id_unique_product_type',
      'value_diff_time_view_user_id_mean',
       'value_diff_time_view_user_id_max', 'value_diff_time_view_user_id_min', 'vc_user_id_mode_item_id', 'vc_user_id_mode_category_1',
       'vc_user_id_mode_category_2', 'vc_user_id_mode_category_3',
       'vc_user_id_mode_product_type', 'gg_1_diff', 'gg_1_ratio', 'gg_2_diff',
       'gg_2_ratio']



param_lgb = {
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'boost': 'gbdt',
    'feature_fraction': 0.8,
    'learning_rate': 0.01,
    'metric':'auc',
    'num_leaves': 31,
    'num_threads': 8,
    'objective': 'binary',
#     'lambda_l1':1
#     'lambda_l2':30
}



kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 228)


pred = pd.DataFrame()
score = []

for i , (train_index, test_index) in enumerate(kf.split(data.loc[:ltr-1, :], data.loc[:ltr-1, 'is_click'])):
    tr = lgb.Dataset(np.array(data[train_cols])[train_index], np.array(data['is_click'])[train_index])
    te = lgb.Dataset(np.array(data[train_cols])[test_index], np.array(data['is_click'])[test_index],
                     reference=tr)
    bst = lgb.train(param_lgb, tr, num_boost_round=10000, 
            valid_sets=te, early_stopping_rounds=int(5 / param_lgb['learning_rate']), verbose_eval = 100)

    pred[str(i)] = bst.predict(np.array(data[train_cols])[ltr:, :])



ans = pd.DataFrame()
ans['impression_id'] = data.loc[ltr:, 'impression_id'].values
ans['is_click'] = (pred.rank().mean(axis = 1) / len(pred)).values



ans.to_csv('answer.csv', index = None)